
**Рейтинг заявок користувачів**

In [1]:
# Завантаження бібліотек
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# завантаження таблиць з даними
df = pd.read_csv(r'C:\Users\illor\Desktop\Python\applications(2.0).csv')
df.head()


,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3


In [3]:
df_ind = pd.read_csv(r'C:\Users\illor\Desktop\Python\industries(2.0).csv')
df_ind.head()

,Industry,Score
0,Blockchain,0
1,Public services / Government,20
2,Adtech / Advertising,10
3,Telecom,15
4,Automotive,15


In [4]:
# візуальний аналіз даних ( в тому числі за допомогою extensions "Data Wrangler")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13315 entries, 0 to 13314
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applied at       13315 non-null  object 
 1   Amount           13272 non-null  float64
 2   Age              13315 non-null  int64  
 3   Gender           13315 non-null  object 
 4   Industry         13315 non-null  object 
 5   Marital status   13315 non-null  object 
 6   External Rating  13243 non-null  float64
 7   Education level  13282 non-null  object 
 8   Location         11540 non-null  object 
 9   applicant_id     13315 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.0+ MB


# Підготовка даних 

In [5]:
# видаляємо рядки, які мають дублікати applicant_id (залишаємо останній рядок з дублікатів )
df = df.drop_duplicates(subset='applicant_id', keep='last')

# В полі 'External Rating' заповни відсутні значення нулями (враховуючи, що рейтинг дорівнює нулю, якщо External Rating' дорівнює нулю  та,
# згідно поставленій задачі результуюча таблиця повинна мати лише заявки з рейтингом більше нуля, то
# видаляємо з таблиці рядки, які в полі 'External Rating' не мають даних)

# видаляю рядки , в яких відсутнє значення 'Amount' ( їх рейтинг дорівнює нулю )

df = df.dropna(subset=['Amount', 'External Rating'])

# в полі 'Education level' заповнюю відсутні значення текстом “Середня”
df['Education level'] = df['Education level'].fillna('Середня')

# додаю з файлу industries.csv, а саме, рейтинги індустрій (df_ind).
df = pd.merge(df, df_ind, on='Industry', how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13173 entries, 0 to 13172
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applied at       13173 non-null  object 
 1   Amount           13173 non-null  float64
 2   Age              13173 non-null  int64  
 3   Gender           13173 non-null  object 
 4   Industry         13173 non-null  object 
 5   Marital status   13173 non-null  object 
 6   External Rating  13173 non-null  float64
 7   Education level  13173 non-null  object 
 8   Location         11440 non-null  object 
 9   applicant_id     13173 non-null  object 
 10  Score            13173 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1.1+ MB


In [ ]:
# видаляю рядки, де поле 'External Rating'  дорівнює нулю

df = df[df['External Rating'] != 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12722 entries, 0 to 13172
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applied at       12722 non-null  object 
 1   Amount           12722 non-null  float64
 2   Age              12722 non-null  int64  
 3   Gender           12722 non-null  object 
 4   Industry         12722 non-null  object 
 5   Marital status   12722 non-null  object 
 6   External Rating  12722 non-null  float64
 7   Education level  12722 non-null  object 
 8   Location         11034 non-null  object 
 9   applicant_id     12722 non-null  object 
 10  Score            12722 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1.2+ MB


# Розрахунок рейтингу 

Правила розрахунку рейтингу:

Рейтинг має бути числом від 0 до 100

Рейтинг - сума оцінок заявки по 6 критеріям.

Рейтинг дорівнює нулю, якщо відсутнє значення 'Amount' або якщо 'External Rating' дорівнює нулю **(враховано при підготовці даних)**

Якщо вік заявника між 35 та 55, до рейтингу додається 20 балів

Якщо заявка була подана не у вихідні, до рейтингу додається 20 балів

Якщо заявника одружений, до рейтингу додається 20 балів

Якщо заявника знаходиться в Києві чи області, до рейтингу додається 10 балів 

Значення 'Score' з таблиці industries.csv також додається до заявки (і складає від 0 до 20 балів)

Якщо 'External Rating' більше чи дорівнює 7, до рейтингу додається 20 балів

Якщо 'External Rating' менше чи дорівнює 2, з рейтингу віднімається 20 балів


In [7]:
# конвертація Dtype object Dtype Datetime
# df['Applied at'] = pd.to_datetime(df['Applied at'],errors='coerce')

df['Applied at'] = pd.to_datetime(df['Applied at'], format='mixed')

In [8]:
df['Age_R'] = ((df['Age'] >= 35) & (df['Age'] <= 55)) * 20
df['Applied at_R'] = (df['Applied at'].dt.weekday < 5) * 20
df['Status'] = (df['Marital status'] == 'Married') * 20
df['Location_R'] = (df['Location'] == 'Київ чи область') * 10

# Створення функції для обчислення балів рейтингу в залежності від значення 'External Rating'
def calculate_Exrating(row):
    rating = 0

    # Якщо 'External Rating' >= 7, додаємо 20 балів
    if row['External Rating'] >= 7:
        rating += 20
    # Якщо 'External Rating' <= 2, віднімаємо 20 балів
    elif row['External Rating'] <= 2:
        rating -= 20
    return rating

# Створення нового стовпця 'ExRating' з використанням функції
df['ExRating'] = df.apply(calculate_Exrating, axis=1)

# Розраховуємо загальний рейтинг
df['Rating'] = df['Age_R'] + df['Applied at_R'] + df['Status'] + df['Location_R'] + df['ExRating'] + df['Score']

# Фільтруємо результат (тільки ті рядки, де рейтинг від 0 !!!!!!)
df = df[(df['Rating'] > 0) ].reset_index(drop=True)

df

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score,Age_R,Applied at_R,Status,Location_R,ExRating,Rating
0,2022-11-30 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10,0,20,0,10,0,40
1,2022-11-30 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15,0,20,0,0,0,35
2,2022-11-30 10:27:38,16500.0,31,Чоловік,E-commerce,Single,8.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,f720bf9c5c4c3e10a8568c1699847696,15,0,20,0,10,20,65
3,2022-11-30 10:27:42,4200.0,30,Чоловік,Media,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,0aaf59fb3ef90f50ccd4800312e5c271,5,0,20,20,10,-20,35
4,2022-11-30 10:27:44,3600.0,24,Чоловік,E-commerce,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,aa54d6e672e4233febadbafdfd048327,15,0,20,20,0,-20,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12553,2023-01-09 11:01:00,12000.0,25,Чоловік,Medtech / Healthcare,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Харків чи область,01908b949a6a1eb44d1ee013ec377f9d,15,0,20,0,0,0,35
12554,2023-01-09 11:14:00,10500.0,28,Чоловік,Blockchain,Married,5.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,6599ba629cfcfe8ac42c7d31c13f62de,0,0,20,20,0,0,40
12555,2023-01-09 11:19:00,5790.0,25,Чоловік,Blockchain,Married,3.0,"Вища (бакалавр, спеціаліст, магістр)",Дніпро чи область,e57f181bec5b11870a541c43d9b960f7,0,0,20,20,0,0,40
12556,2023-01-09 11:28:00,13500.0,31,Чоловік,Mobile,Single,8.0,Середня,Одеса чи область,a340c62a1226509949ef60df10a7d726,15,0,20,0,0,20,55


In [9]:
print ('Прийняті заявки : \n\n ' ,df['applicant_id'])

Прийняті заявки : 

  0        10dbafaeb46c09e96b6987c03bbb3498
1        5d21f3795b50de8e8f8f8d5f48b754f3
2        f720bf9c5c4c3e10a8568c1699847696
3        0aaf59fb3ef90f50ccd4800312e5c271
4        aa54d6e672e4233febadbafdfd048327
                       ...               
12553    01908b949a6a1eb44d1ee013ec377f9d
12554    6599ba629cfcfe8ac42c7d31c13f62de
12555    e57f181bec5b11870a541c43d9b960f7
12556    a340c62a1226509949ef60df10a7d726
12557    a36a922c50b554cad6cd0eb98a5eaad8
Name: applicant_id, Length: 12558, dtype: object


# Cередній рейтинг прийнятих заявок у кожен тиждень

In [10]:
# Виводимо номер тижня
df['Week'] = df['Applied at'].dt.isocalendar().week

weekly_avg_rating = df.groupby('Week')['Rating'].agg('mean').round(2)


print(f'Середній рейтинг прийнятих заявок: {weekly_avg_rating}')

Середній рейтинг прийнятих заявок: Week
1     51.23
2     51.88
48    50.74
49    49.52
50    49.90
51    47.02
52    50.98
Name: Rating, dtype: float64
